<a href="https://colab.research.google.com/github/gitriyer/StockPrediction/blob/main/StockPrediction_NeuralNetworks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#%%
import numpy as np
import pandas as pd
import datetime as dt
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
import plotly.graph_objs as go

# Load Data
company = 'GOOGL'

start = dt.datetime(2015, 1, 1)
end = dt.datetime(2023, 1, 1)

data = yf.download(company, start=start, end=end)

# Prepare Data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data['Close'].values.reshape(-1, 1))

prediction_days = 60

x_train = []
y_train = []

for x in range(prediction_days, len(scaled_data)):
    x_train.append(scaled_data[x-prediction_days:x, 0])
    y_train.append(scaled_data[x, 0])

x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

# Build the Model
model = Sequential()

model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=1))  # Prediction of closing price

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_train, y_train, epochs=25, batch_size=32)  # model will see same data 25 times, and 32 units at once

# Test model accuracy on existing data
test_start = dt.datetime(2023, 1, 1)
test_end = dt.datetime.now()

test_data = yf.download(company, start=test_start, end=test_end)
actual_prices = test_data['Close'].values

total_dataset = pd.concat((data['Close'], test_data['Close']), axis=0)

model_inputs = total_dataset[len(total_dataset) - len(test_data) - prediction_days:].values
model_inputs = model_inputs.reshape(-1, 1)

model_inputs = scaler.fit_transform(model_inputs)

# Make predictions on test data
x_test = []

for x in range(prediction_days, len(model_inputs)):
    x_test.append(model_inputs[x-prediction_days:x, 0])

x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

predicted_prices = model.predict(x_test)
predicted_prices = scaler.inverse_transform(predicted_prices)

# Plot test predictions using Plotly
fig = go.Figure()

fig.add_trace(go.Scatter(x=test_data.index, y=actual_prices, mode='lines', name=f'Actual {company} Price', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=test_data.index, y=predicted_prices.flatten(), mode='lines', name=f'Predicted {company} Price', line=dict(color='red')))

fig.update_layout(
    title=f'{company} Share Price',
    xaxis_title='Time',
    yaxis_title=f'{company} Share Price',
    legend=dict(x=0, y=1, traceorder='normal'),
)

fig.show()

# Predict Next Day Stock Price
real_data = [model_inputs[len(model_inputs) + 1 - prediction_days:len(model_inputs + 1), 0]]
real_data = np.array(real_data)
real_data = np.reshape(real_data, (real_data.shape[0], real_data.shape[1], 1))

prediction = model.predict(real_data)
prediction = scaler.inverse_transform(prediction)
print(f"Prediction : {prediction}")


[*********************100%%**********************]  1 of 1 completed
/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/25
62/62 ━━━━━━━━━━━━━━━━━━━━ 13s 75ms/step - loss: 0.0346
Epoch 2/25
62/62 ━━━━━━━━━━━━━━━━━━━━ 5s 82ms/step - loss: 0.0040
Epoch 3/25
62/62 ━━━━━━━━━━━━━━━━━━━━ 10s 75ms/step - loss: 0.0028
Epoch 4/25
62/62 ━━━━━━━━━━━━━━━━━━━━ 6s 97ms/step - loss: 0.0028
Epoch 5/25
62/62 ━━━━━━━━━━━━━━━━━━━━ 9s 77ms/step - loss: 0.0031
Epoch 6/25
62/62 ━━━━━━━━━━━━━━━━━━━━ 6s 101ms/step - loss: 0.0027
Epoch 7/25
62/62 ━━━━━━━━━━━━━━━━━━━━ 9s 84ms/step - loss: 0.0023
Epoch 8/25
62/62 ━━━━━━━━━━━━━━━━━━━━ 10s 77ms/step - loss: 0.0025
Epoch 9/25
62/62 ━━━━━━━━━━━━━━━━━━━━ 6s 98ms/step - loss: 0.0020
Epoch 10/25
62/62 ━━━━━━━━━━━━━━━━━━━━ 5s 76ms/step - loss: 0.0021
Epoch 11/25
62/62 ━━━━━━━━━━━━━━━━━━━━ 5s 88ms/step - loss: 0.0023
Epoch 12/25
62/62 ━━━━━━━━━━━━━━━━━━━━ 9s 75ms/step - loss: 0.0020
Epoch 13/25
62/62 ━━━━━━━━━━━━━━━━━━━━ 7s 100ms/step - loss: 0.0022
Epoch 14/25
62/62 ━━━━━━━━━━━━━━━━━━━━ 9s 76ms/step - loss: 0.0020
Epoch 15/25
62/62 ━━━━━━━━━━━━━━━━━━━━ 6s 99ms/step - loss: 0.0018

[*********************100%%**********************]  1 of 1 completed


13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 84ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 508ms/step
Prediction : [[166.24367]]
